# QA over SQL data

## Intro
* We will create a Q&A app over tabular data in databases.
* These app will allow us to **ask a question about the data in a database in natural language and get back an answer also in natural language**.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

**Connect with an LLM**

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

#### Connect with the database
* In this example we will use a SQLite connection with the `street_tree_db` database we have in the `data` folder.
* **Remember to download the data folder from the Github repository**.
* The `data` folder must be in the root directory of this app.
* We can interface with the database using the SQLAlchemy-driven `SQLDatabase` class:

In [3]:
from langchain_community.utilities import SQLDatabase

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

We can create a simple chain that takes a question and does the following:
* Convert the question into a SQL query;
* Execute the query;
* Use the result to answer the original question.

The first step in a SQL chain is to take the user input and convert it to a SQL query. LangChain comes with a **built-in chain** for this, `create_sql_query_chain`:

#### Step 1: Translating a question in natural language into an SQL query.

In [4]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

response = chain.invoke({"question": "How many species of trees are in San Francisco?"})

response

'SELECT COUNT(DISTINCT "qSpecies") AS TotalSpecies\nFROM street_trees;'

* We can execute the query to make sure it's valid:

In [5]:
db.run(response)

'[(148,)]'

* We can also inspect the chain directly for its prompts. Pay attention to the second paragraph, where it says: **"Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database."** This is a limitation and if it is not handled well can cause errors.

In [6]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

#### Step 2: Executing the SQL query.

* Now that we've generated a SQL query, we'll want to execute it.
* When you are in production, consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution.
* We can **use the QuerySQLDatabaseTool to easily add query execution to our chain**:
    * The user asks a question.
    * The write_query chain has que question as input and the SQL query as output.
    * The execute_query chain has the SQL query as input and the SQL query execution as output.

In [7]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)

execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "List the species of trees that are present in San Francisco"})

'[("Arbutus \'Marina\' :: Hybrid Strawberry Tree",), (\'Afrocarpus gracilior :: Fern Pine\',), ("Thuja occidentalis \'Emerald\' :: Emerald Arborvitae",), ("Magnolia grandiflora \'Little Gem\' :: Little Gem Magnolia",), (\'Platanus x hispanica :: Sycamore: London Plane\',), (\'Ulmus parvifolia :: Chinese Elm\',), (\'Leptospermum laevigatum :: Australian Tea Tree\',), ("Olea europaea \'Fruitless\' :: Fruitless Olive",), (\'Ficus rubiginosa :: Port Jackson Fig\',), (\'Cycas revoluta :: Sago palm\',), (\'Lophostemon confertus :: Brisbane Box\',), (\'Podocarpus gracilor :: Fern Pine\',), (\'Eriobotrya deflexa :: Bronze Loquat\',), (\'Ligustrum lucidum :: Glossy Privet\',), (\'Syagrus romanzoffianum :: Queen Palm\',), ("Ficus microcarpa nitida \'Green Gem\' :: Indian Laurel Fig Tree \'Green Gem\'",), (\'Prunus cerasifera :: Cherry Plum\',), (\':: To Be Determine\',), (\'Myoporum laetum :: Myoporum\',), (\'Tristaniopsis laurina :: Swamp Myrtle\',), (\'Ginkgo biloba :: Maidenhair Tree\',), (\'

#### Step 3: Translate the SQL response into a natural language response

* Now that we've got a way to generate and execute queries, we need to **combine the original question and SQL query result with the chat model to generate a final answer in natural language**.
* We can do this by passing question and result to the LLM like this:

In [8]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

sqlite_db_path = "data/street_tree_db.sqlite"

db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, 
    corresponding SQL query, and SQL result, 
    answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

write_query = create_sql_query_chain(llm, db)

execute_query = QuerySQLDataBaseTool(db=db)

#chain = write_query | execute_query

#chain.invoke({"question": "List the species of trees that are present in San Francisco"})

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "List the species of trees that are present in San Francisco"})

"The species of trees present in San Francisco include Arbutus 'Marina' Hybrid Strawberry Tree, Afrocarpus gracilior Fern Pine, Thuja occidentalis 'Emerald' Emerald Arborvitae, Magnolia grandiflora 'Little Gem' Little Gem Magnolia, Platanus x hispanica Sycamore: London Plane, Ulmus parvifolia Chinese Elm, Leptospermum laevigatum Australian Tea Tree, Olea europaea 'Fruitless' Fruitless Olive, Ficus rubiginosa Port Jackson Fig, Cycas revoluta Sago palm, Lophostemon confertus Brisbane Box, Podocarpus gracilor Fern Pine, Eriobotrya deflexa Bronze Loquat, Ligustrum lucidum Glossy Privet, Syagrus romanzoffianum Queen Palm, Ficus microcarpa nitida 'Green Gem' Indian Laurel Fig Tree 'Green Gem', Prunus cerasifera Cherry Plum, Myoporum laetum Myoporum, Tristaniopsis laurina Swamp Myrtle, Ginkgo biloba Maidenhair Tree, Pyrus calleryana Ornamental Pear, Morus alba 'Fruitless' Fruitless Mulberry, Magnolia grandiflora Southern Magnolia, Pittosporum undulatum Victorian Box, Lyonothamnus floribundus 

#### Let's review what is happening in the above chain.
* The user asks a question (identified by the variable name "question").
* We use RunnablePassthrough to get that "question" variable, and we use .assign() twice to get the other two variables required by the prompt template: "query" and "result".
* With the first .assign(), the write_query chain has que question as input and the SQL query (identified by the variable name "query") as output.
* With the second .assign(), the execute_query chain has the SQL query as input and the SQL query execution (identified by the variable name "result") as output.
* The prompt template has the question (identified by the variable name "question"), the SQL query (identified by the variable name "query") and the SQL query execution (identified by the variable name "result") as input, and the final prompt as the output.
* The chat model has the prompt as he input and the AIMessage with the response in natural language as the output.
* The StrOutputParser has the AIMessage with the response in natural language as the input and the response in natural language as a string of text as the output.

#### About the role of .assign() in this chain
In this exercise we have learned more about the .assign() built-in method for Runnables. We have seen that the .assign() method **allows us to include additional keys and values** based on existing input.
* With the first .assign(), the write_query chain has the question as input and the SQL query (identified by the variable name "query") as output.
* With the second .assign(), the execute_query chain has the SQL query as input and the SQL query execution (identified by the variable name "result") as output.

The .assign() method allows us to add new keys and values to the output of a Runnable **while keeping the original keys and values intact**. See the process again:
* We use RunnablePassthrough to get that "question" variable, and we use .assign() twice to get the other two variables required by the prompt template: "query" and "result".
* With the first .assign(), the write_query chain has que question as input and the SQL query (identified by the variable name "query") as output.
* With the second .assign(), the execute_query chain has the SQL query as input and the SQL query execution (identified by the variable name "result") as output.